In [1]:
from agents import Agent, Runner, TResponseInputItem

simple_agent = Agent(
    name="Bob",
    instructions="You are a friendly assistant. Reply concisely."
)

convo: list[TResponseInputItem] = [
    {"content": "Hey!", "role": "user"},
    {"content": "Hey! How are you? How can I help you today?", "role": "assistant"},
    {"content": "Im having a bad day", "role": "user"}
]
result = await Runner.run(simple_agent, convo)
for message in result.to_input_list():
    print(message)

{'content': 'Hey!', 'role': 'user'}
{'content': 'Hey! How are you? How can I help you today?', 'role': 'assistant'}
{'content': 'Im having a bad day', 'role': 'user'}
{'id': 'msg_67e868b45444819181be3be2e97916920b5e95bb26736f2a', 'content': [{'annotations': [], 'text': "I'm sorry to hear that. Do you want to talk about what's going on?", 'type': 'output_text'}], 'role': 'assistant', 'status': 'completed', 'type': 'message'}


In [3]:
convo: list[TResponseInputItem] = []
print("You are now chatting with agent Bob. Type 'exit' to end the conversation.")
while True:
    user_input = input("You: ")
    print("You: " + user_input)

    if user_input == "exit":
        print("Goodbye!")
        break

    convo.append({"content": user_input, "role": "user"})
    result = await Runner.run(simple_agent, convo)

    print(f"Agent: {result.final_output}")

    convo = result.to_input_list()

You are now chatting with agent Bob. Type 'exit' to end the conversation.
You: hey
Agent: Hey there! How can I assist you today?
You: exit
Goodbye!


In [8]:
from agents import Agent, Runner, handoff, RunContextWrapper

history_tutor_agent = Agent(
    name="History Tutor",
    handoff_description="Specialist agent for historical questions",
    instructions="You provide assistance with historical queries. Explain important events and context clearly.",
)

math_tutor_agent = Agent(
    name="Math Tutor",
    handoff_description="Specialist agent for math questions",
    instructions="You provide assistance with math queries. Explain your reasoning at each step and include examples"
)

def on_math_handoff(ctx: RunContextWrapper[None]):
    print("Handing off to math tutor agent")

# This agent has the capability to handoff to either the history or math tutor agent
triage_agent = Agent(
    name="Triage Agent",
    instructions="You determine which agent to use based on the user's homework question." +
    "If neither agent is relevant, provide a general response.",
    handoffs=[history_tutor_agent, handoff(math_tutor_agent, on_handoff=on_math_handoff)]
)

convo: list[TResponseInputItem] = []
last_agent = triage_agent
print("You are now chatting with the triage agent. Type 'exit' to end the conversation.")
while True:
    user_input = input("You: ")
    print("You: " + user_input)

    if user_input == "exit":
        print("Goodbye!")
        break

    convo.append({"content": user_input, "role": "user"})
    result = await Runner.run(last_agent, convo)

    convo = result.to_input_list()
    last_agent = result.last_agent

    print(f"{last_agent.name}: {result.final_output}\n")

You are now chatting with the triage agent. Type 'exit' to end the conversation.
You: hey
Triage Agent: Hello! How can I assist you today? Do you have a homework question or something else on your mind?

You: help me learn how to divide
Handing off to math tutor agent
Math Tutor: Of course! Let’s go through the basics of division and look at some examples.

### What is Division?

Division is the process of splitting a number (called the dividend) into equal parts. The number you divide by is called the divisor, and the result is the quotient. Sometimes, there may be a remainder.

### Example 1: Simple Division

Let's divide 20 by 4.

1. **Dividend**: 20
2. **Divisor**: 4

To find the **Quotient**, think of how many times 4 fits into 20 exactly. 

\[ 4 \times 5 = 20 \]

So, 20 divided by 4 equals 5. Here, the quotient is 5, and there is no remainder.

### Example 2: Division with a Remainder

Now, let’s divide 22 by 4.

1. **Dividend**: 22
2. **Divisor**: 4

Check how many times 4 fits 